In [4]:
import arcpy

arcpy.env.workspace = "C:\\Users\\Chima Okoli\\OneDrive\\Desktop\\GEOG592\\Lab7"
arcpy.env.overwriteOutput = True

Cntyshp = 'Counties.shp'
rankFile = open('rank.txt', 'w')

#Create an if not statement to add the field "rank1".
if not arcpy.ListFields(Cntyshp, 'rank1'):
    arcpy.AddField_management(Cntyshp, 'rank1', 'SHORT')

recList = []   #Create an empty record list to add the list
recNum = 0     #Create a record number to track the records from the field in the shapefile

with arcpy.da.SearchCursor(Cntyshp, ['2000']) as cursor:  #Create a search cursor
    for row in cursor:
        recList.append([recNum, row[0], 0])    #the record number keeps track of each record with county names while the last number (0) keeps track of the ranking result.
        recNum += 1

sortedRecList = sorted(recList, key = lambda x:x[1], reverse = True) #sorting based on the 'row[0]' list item and reversing the sorting order to descending

#Sort the ranking result
rank = 1
for aRec in sortedRecList:
    aRec[2] = rank #picks up the '0' record in the list which is the ranking result and ranks them from 1,2,3,...
    rank += 1

#Sort the results back to the original order based on the object ID number(how it was in the original table but with the ranks reflecting population descending)
regRecList = sorted(sortedRecList, key = lambda x:x[0])

#Create another search cursor to grab the county name, state name and the population field so you can output to a textfile.
with arcpy.da.SearchCursor(Cntyshp, ['NAME', 'STATE_NAME', '2000']) as cursor:
    recNum = 0
    for row in cursor:
        r = regRecList[recNum][2]    #The recNum picks the first record and the third value which is the ranking number from regRecList
        recNum += 1                  #The recNum increments to pick all ranking results one after the other.
        outString = row[0] + ',' + row[1] + ','+ str(row[2]) + ',' + str(r) + '\n'      #row[0] is NAME, row[1] is STATE_NAME, row[2] is the 2000 while 'r' is the rank result.
        rankFile.write(outString)
        
        
#for lab 7, use update cursor to add the rank field. You don't need the last 2 rows in the search cursor above.

#for the RMI, you need to have two parameter feilds 2 (1990) & 3 (2000) for the tool. The recLIst.append will now be recLIst.append([recNum, row[0], row[1], 0, 0])

#The sortedRecLIst and the for loop for the sortedRecList will be done twice for each rank.
#in the if not statement, you can change the "rank1" to RMI.
#r1 should be parameter 3 while r2 should be parameter 2.

rankFile.close()

In [ ]:
import arcpy

class RankTool(object):
    def __init__(self):
        self.label = "Rank Tool"
        self.description = "Rank a numerical field and add a new rank field to the input feature class."
        self.canRunInBackground = False

    def getParameterInfo(self):
        # Input feature class parameter
        in_fc = arcpy.Parameter(
            displayName="Input Feature Class",
            name="in_fc",
            datatype="DEFeatureClass",
            parameterType="Required",
            direction="Input")

        # Numerical field parameter
        in_field = arcpy.Parameter(
            displayName="Numerical Field to Rank",
            name="in_field",
            datatype="Field",
            parameterType="Required",
            direction="Input",
            parameterDependencies=["in_fc"])

        # New field name parameter
        out_field_name = arcpy.Parameter(
            displayName="New Rank Field Name",
            name="out_field_name",
            datatype="GPString",
            parameterType="Required",
            direction="Input")

        parameters = [in_fc, in_field, out_field_name]
        return parameters

    def execute(self, parameters, messages):
        # Get input parameters
        in_fc = parameters[0].valueAsText
        in_field = parameters[1].valueAsText
        out_field_name = parameters[2].valueAsText

        # Set environment variables
        arcpy.env.workspace = arcpy.Describe(in_fc).path
        arcpy.env.overwriteOutput = True

        # Check if output field already exists
        if out_field_name in [f.name for f in arcpy.ListFields(in_fc)]:
            messages.addWarning("Output field already exists. Overwriting existing values.")

        # Add new field for rank
        arcpy.AddField_management(in_fc, out_field_name, "SHORT")

        # Populate rank values
        with arcpy.da.UpdateCursor(in_fc, [in_field, out_field_name]) as cursor:
            recList = []
            recNum = 0
            for row in cursor:
                recList.append([recNum, row[0], 0])
                recNum += 1
            sortedRecList = sorted(recList, key=lambda x: x[1], reverse=True)
            for rank, aRec in enumerate(sortedRecList):
                cursor.updateRow([aRec[1], rank+1])

        messages.addMessage("Ranking complete.")

        return


In [ ]:
# -*- coding: utf-8 -*-

import arcpy


class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = " My Second Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool]


class Tool(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Rank Tool"
        self.description = "Rank a numerical field from an input dataset and add a new rank field to the input feature class."
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName="Input Feature Class",name="in_fc",datatype="DEFeatureClass",parameterType="Required",direction="Input")
        param1 = arcpy.Parameter(displayName="Numerical Field to Rank",name="in_field",datatype="Field",parameterType="Required",direction="Input")
        param1.parameterDependencies = [param0.name]
        param2 = arcpy.Parameter(displayName="New Rank Field Name",name="out_field",datatype="GPString",parameterType="Required",direction="Input")
        params = [param0, param1, param2]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        in_fc = parameters[0].valueAsText
        in_field = parameters[1].valueAsText
        out_field = parameters[2].valueAsText

        arcpy.env.workspace = "C:\\Users\\Chima Okoli\\OneDrive\\Desktop\\GEOG592\\Lab7"
        arcpy.env.overwriteOutput = True

        #Cntyshp = 'Counties.shp'
        #rankFile = open('rank.txt', 'w')

        #if out_field in [f.name for f in arcpy.ListFields(in_fc)]:
            #arcpy.addMessage("Output field already exists. Overwriting existing values.")
        

        #Create an if not statement to add the field "rank1".
        if not arcpy.ListFields(in_fc, out_field):
            arcpy.AddField_management(in_fc, out_field, "LONG")

        recList = []   
        recNum = 0     

        with arcpy.da.SearchCursor(in_fc, [in_field]) as cursor:  
            for row in cursor:
                recList.append([recNum, row[0], 0])    
                recNum += 1

        sortedRecList = sorted(recList, key = lambda x:x[1], reverse = True) 

        #Sort the ranking result
        rank = 1
        for aRec in sortedRecList:
            aRec[2] = rank 
            rank += 1

        rankDict = {}
        for aRec in sortedRecList:
            rankDict[aRec[0]] = aRec[2]

        # Create an update cursor to update the values in the new field
        with arcpy.da.UpdateCursor(in_field, [out_field]) as cursor:
            for row in cursor:
                recNum = cursor.currentRow[0]
                row[0] = rankDict[recNum]
                cursor.updateRow(row)

        
        #regRecList = sorted(sortedRecList, key = lambda x:x[0])

        #with arcpy.da.UpdateCursor(in_fc, [in_field, out_field]) as cursor:
            #recNum = 0
            #for row in cursor:
                #r = regRecList[recNum][2]    
                #recNum += 1
                #cursor.updateRow([r, rank+1])

        #with arcpy.da.UpdateCursor(in_fc, [in_field, out_field]) as cursor:
            #recList = []
            #recNum = 0
            #for row in cursor:
                #recList.append([recNum, row[0], 0])
                #recNum += 1
            #sortedRecList = sorted(recList, key=lambda x: x[1], reverse=True)
            #for rank, aRec in enumerate(sortedRecList):
                #cursor.updateRow([aRec[1], rank+1])

        arcpy.AddMessage("Ranking complete.")

        
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return
